In [0]:

import boto3
import numpy as np
import pickle
import argparse
import ast
try:
	import ember
except:
	os.system('wget https://github.com/endgameinc/ember/archive/master.zip')
	os.system('unzip master.zip')
	os.system('rm master.zip')
	os.system('cp -r ember-master/* .')
	os.system('rm -r ember-master')
	os.system('pip install -r requirements.txt')
	os.system('python setup.py install')
	import ember
import json
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
def main():
	prog = "predict_sample"
	descr = "Using the model to predict a single PE's binary."
	parser = argparse.ArgumentParser(prog=prog, description=descr)
	parser.add_argument("-v", "--featureversion", type=int, default=2, help="EMBER feature version")
	parser.add_argument("binaries", metavar="BINARIES", type=str, nargs="+", help="PE files to classify")
	args = parser.parse_args()

	extractor = ember.PEFeatureExtractor(args.featureversion)
	data = open(args.binaries[0],'rb').read()
	data = extractor.feature_vector(data)
	data = np.array(data, dtype=np.float32)
	data = ss.transform([sample_data])
	data = np.reshape(data,(-1,1,2381))
	data = data.tolist()


	client = boto3.client('runtime.sagemaker', 
    region_name='us-east-1', # make sure to set correct region
    aws_access_key_id=key id, # These you get from AWS, for your account
    aws_secret_access_key=secret accesss key)
 
	response = client.invoke_endpoint(EndpointName='sagemaker-tensorflow-2020-04-28-00-12-37-103' , Body=json.dumps(data))
	response_body = response['Body']
	bbtyes = response_body.read()
	astr = bbtyes.decode("UTF-8")
	d = ast.literal_eval(astr)
	d = d['outputs']['score']['floatVal']

	if d[0] >=.5:
		print("Malicious")
	else:
		print("Benign")
  